<a href="https://colab.research.google.com/github/Riyad2k9/Riyad/blob/master/Hyperparameter_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Md Riyad Hossain
##SID: 20535580
###Project: Hyperparamter optimization

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.svm import SVC,SVR
from sklearn import datasets
import scipy.stats as stats

In [2]:
import pandas as pd
df=pd.read_excel('DPTL Fiber Test Report.xlsx',sheet_name='app')
print(df.columns)
print(df.count())

Index(['Date', 'Denier', 'Max_Breaking_Force_(cN)', 'Min_Breaking_Force_(cN)',
       'Avg_Breaking_Force_(cN)', 'Max_Elongation_( %)', 'Min_Elongation_( %)',
       'Avg_Elongation_( %)', 'Draw Ratio', 'tenacity'],
      dtype='object')
Date                       222
Denier                     222
Max_Breaking_Force_(cN)    222
Min_Breaking_Force_(cN)    222
Avg_Breaking_Force_(cN)    222
Max_Elongation_( %)        222
Min_Elongation_( %)        222
Avg_Elongation_( %)        222
Draw Ratio                 222
tenacity                   222
dtype: int64


In [5]:
#df.head()
X=df.drop(columns=['Date']).values
y= df.tenacity
print(df.columns)

Index(['Date', 'Denier', 'Max_Breaking_Force_(cN)', 'Min_Breaking_Force_(cN)',
       'Avg_Breaking_Force_(cN)', 'Max_Elongation_( %)', 'Min_Elongation_( %)',
       'Avg_Elongation_( %)', 'Draw Ratio', 'tenacity'],
      dtype='object')


#1:Regressors with Default Hyperparameters

Random Forest

In [21]:
clf = RandomForestRegressor()
scores = cross_val_score(clf, X, y, cv=3,scoring='neg_mean_squared_error') # 3-fold cross-validation
print("MSE:"+ str(-scores.mean()))

MSE:0.44817502234233614


Support Vector Machine

In [22]:
#SVM
clf = SVR(gamma='scale')
scores = cross_val_score(clf, X, y, cv=3,scoring='neg_mean_squared_error')
print("MSE:"+ str(-scores.mean()))

MSE:36.53649492107039


K-Nearest Neighbours

In [23]:
#KNN
clf = KNeighborsRegressor()
scores = cross_val_score(clf, X, y, cv=3,scoring='neg_mean_squared_error')
print("MSE:"+ str(-scores.mean()))

MSE:31.00693801801801


Artificial Neural Network

In [26]:
#ANN
from keras.models import Sequential, Model
from keras.layers import Dense, Input
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping
def ANN(optimizer = 'adam',neurons=32,batch_size=32,epochs=50,activation='relu',patience=5,loss='mse'):
    model = Sequential()
    model.add(Dense(neurons, input_shape=(X.shape[1],), activation=activation))
    model.add(Dense(neurons, activation=activation))
    model.add(Dense(1))
    model.compile(optimizer = optimizer, loss=loss)
    early_stopping = EarlyStopping(monitor="loss", patience = patience)# early stop patience
    history = model.fit(X, y,
              batch_size=batch_size,
              epochs=epochs,
              callbacks = [early_stopping],
              verbose=0) #verbose set to 1 will show the training process
    return model

In [27]:
clf = KerasRegressor(build_fn=ANN, verbose=0)
scores = cross_val_score(clf, X, y, cv=3,scoring='neg_mean_squared_error')
print("MSE:"+ str(-scores.mean()))

MSE:7.137122322406706


# Grid Search

1. Random Forest

In [34]:
#Random Forest
from sklearn.model_selection import GridSearchCV
# Define the hyperparameter configuration space
rf_params = {
    'n_estimators': [10, 20, 30],
    'max_features': ['sqrt',0.5],
    'max_depth': [15,20,30,50],
    'min_samples_leaf': [1,2,4,8],
    "bootstrap":[True,False],
    "criterion":['mse','mae']
}
clf = RandomForestRegressor(random_state=0)
grid = GridSearchCV(clf, rf_params, cv=3, scoring='neg_mean_squared_error')
grid_results = grid.fit(X, y)
#grid.fit(X, y)
print(grid.best_params_)
print("MSE:"+ str(-grid.best_score_))

{'bootstrap': False, 'criterion': 'mae', 'max_depth': 15, 'max_features': 0.5, 'min_samples_leaf': 1, 'n_estimators': 30}
MSE:1.0529643617367346


Support Vector Machine

In [35]:
#SVM
from sklearn.model_selection import GridSearchCV
rf_params = {
    'C': [1,10, 100],
    "kernel":['poly','rbf','sigmoid'],
    "epsilon":[0.01,0.1,1]
}
clf = SVR(gamma='scale')
grid = GridSearchCV(clf, rf_params, cv=3, scoring='neg_mean_squared_error')
grid.fit(X, y)
print(grid.best_params_)
print("MSE:"+ str(-grid.best_score_))

{'C': 100, 'epsilon': 0.01, 'kernel': 'rbf'}
MSE:0.9265117164053054


KNN

In [36]:
#KNN
from sklearn.model_selection import GridSearchCV
rf_params = {
    'n_neighbors': [2, 3, 5,7,10]
}
clf = KNeighborsRegressor()
grid = GridSearchCV(clf, rf_params, cv=3, scoring='neg_mean_squared_error')
grid.fit(X, y)
print(grid.best_params_)
print("MSE:"+ str(-grid.best_score_))

{'n_neighbors': 2}
MSE:29.450023761261264


In [37]:
#ANN
from sklearn.model_selection import GridSearchCV
rf_params = {
    'optimizer': ['adam','rmsprop'],
    'activation': ['relu','tanh'],
    'loss': ['mse','mae'],
    'batch_size': [16,32],
    'neurons':[16,32],
    'epochs':[20,50],
    'patience':[2,5]
}
clf = KerasRegressor(build_fn=ANN, verbose=0)
grid = GridSearchCV(clf, rf_params, cv=3,scoring='neg_mean_squared_error')
grid.fit(X, y)
print(grid.best_params_)
print("MSE:"+ str(-grid.best_score_))

{'activation': 'relu', 'batch_size': 16, 'epochs': 50, 'loss': 'mae', 'neurons': 32, 'optimizer': 'adam', 'patience': 5}
MSE:0.4746772858398365


# Random Search 

In [38]:
#SVM
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
rf_params = {
    'C': stats.uniform(0,50),
    "kernel":['poly','rbf','sigmoid'],
    "epsilon":stats.uniform(0,1)
}
n_iter_search=20
clf = SVR(gamma='scale')
Random = RandomizedSearchCV(clf, param_distributions=rf_params,n_iter=n_iter_search,cv=3,scoring='neg_mean_squared_error')
Random.fit(X, y)
print(Random.best_params_)
print("MSE:"+ str(-Random.best_score_))

{'C': 45.097277147474166, 'epsilon': 0.1215945066455495, 'kernel': 'rbf'}
MSE:2.9103970448730103


In [39]:
#KNN
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
rf_params = {
    'n_neighbors': sp_randint(1,20),
}
n_iter_search=10
clf = KNeighborsRegressor()
Random = RandomizedSearchCV(clf, param_distributions=rf_params,n_iter=n_iter_search,cv=3,scoring='neg_mean_squared_error')
Random.fit(X, y)
print(Random.best_params_)
print("MSE:"+ str(-Random.best_score_))

{'n_neighbors': 2}
MSE:29.450023761261264


In [40]:
#ANN
from scipy.stats import randint as sp_randint
from random import randrange as sp_randrange
from sklearn.model_selection import RandomizedSearchCV
rf_params = {
    'optimizer': ['adam','rmsprop'],
    'activation': ['relu','tanh'],
    'loss': ['mse','mae'],
    'batch_size': [16,32,64],
    'neurons':sp_randint(10,100),
    'epochs':[20,50],
    #'epochs':[20,50,100,200],
    'patience':sp_randint(3,20)
}
n_iter_search=10
clf = KerasRegressor(build_fn=ANN, verbose=0)
Random = RandomizedSearchCV(clf, param_distributions=rf_params,n_iter=n_iter_search,cv=3,scoring='neg_mean_squared_error')
Random.fit(X, y)
print(Random.best_params_)
print("MSE:"+ str(-Random.best_score_))

{'activation': 'relu', 'batch_size': 16, 'epochs': 20, 'loss': 'mse', 'neurons': 97, 'optimizer': 'adam', 'patience': 3}
MSE:0.3044385336293817


# Bayesian Optimization with Gaussian Process (BO-GP)

In [45]:
! pip install scikit-optimize

     |████████████████████████████████| 102kB 9.7MB/s 


In [47]:
#SVM
from skopt import Optimizer
from skopt import BayesSearchCV 
from skopt.space import Real, Categorical, Integer
rf_params = {
    'C': Real(0,50),
    "kernel":['poly','rbf','sigmoid'],
    'epsilon': Real(0,1)
}
clf = SVR(gamma='scale')
Bayes = BayesSearchCV(clf, rf_params,cv=3,n_iter=20, scoring='neg_mean_squared_error')
Bayes.fit(X, y)
print(Bayes.best_params_)
print("MSE:"+ str(-Bayes.best_score_))

OrderedDict([('C', 47.226705748385406), ('epsilon', 0.6050391339259759), ('kernel', 'rbf')])
MSE:4.157092337907249


In [48]:
#KNN
from skopt import Optimizer
from skopt import BayesSearchCV 
from skopt.space import Real, Categorical, Integer
rf_params = {
    'n_neighbors': Integer(1,20),
}
clf = KNeighborsRegressor()
Bayes = BayesSearchCV(clf, rf_params,cv=3,n_iter=10, scoring='neg_mean_squared_error')
Bayes.fit(X, y)
print(Bayes.best_params_)
print("MSE:"+ str(-Bayes.best_score_))

OrderedDict([('n_neighbors', 6)])
MSE:31.708766366366362


In [49]:
#ANN
from skopt import Optimizer
from skopt import BayesSearchCV 
from skopt.space import Real, Categorical, Integer
rf_params = {
    'optimizer': ['adam','rmsprop'],
    'activation': ['relu','tanh'],
    'loss': ['mse','mae'],
    'batch_size': [16,32,64],
    'neurons':Integer(10,100),
    'epochs':[20,50],
    #'epochs':[20,50,100,200],
    'patience':Integer(3,20)
}
clf = KerasRegressor(build_fn=ANN, verbose=0)
Bayes = BayesSearchCV(clf, rf_params,cv=3,n_iter=10, scoring='neg_mean_squared_error')
Bayes.fit(X, y)
print(Bayes.best_params_)
print("MSE:"+ str(-Bayes.best_score_))

OrderedDict([('activation', 'relu'), ('batch_size', 16), ('epochs', 31), ('loss', 'mae'), ('neurons', 89), ('optimizer', 'adam'), ('patience', 4)])
MSE:0.3077114453802641


# gp_minimize

In [50]:
#SVM
from skopt.space import Real, Integer
from skopt.utils import use_named_args

reg = SVR(gamma='scale')
space  = [Real(0, 50, name='C'),
          Categorical(['poly','rbf','sigmoid'], name='kernel'),
          Real(0, 1, name='epsilon'),
         ]

@use_named_args(space)
def objective(**params):
    reg.set_params(**params)

    return -np.mean(cross_val_score(reg, X, y, cv=3, n_jobs=-1,
                                    scoring="neg_mean_squared_error"))
from skopt import gp_minimize
res_gp = gp_minimize(objective, space, n_calls=20, random_state=0)
print("MSE:%.4f" % res_gp.fun)
print(res_gp.x)

MSE:3.9821
[42.36258693920628, 'rbf', 0.38438170729269994]


In [51]:
#KNN
from skopt.space import Real, Integer
from skopt.utils import use_named_args

reg = KNeighborsRegressor()
space  = [Integer(1, 20, name='n_neighbors')]

@use_named_args(space)
def objective(**params):
    reg.set_params(**params)

    return -np.mean(cross_val_score(reg, X, y, cv=3, n_jobs=-1,
                                    scoring="neg_mean_squared_error"))
from skopt import gp_minimize
res_gp = gp_minimize(objective, space, n_calls=10, random_state=0)
print("MSE:%.4f" % res_gp.fun)
print(res_gp.x)

MSE:29.4500
[2]
